# Plant Pathology 2020 - FGVC7
Identify the category of foliar diseases in apple trees

Kaggle competition - https://www.kaggle.com/c/plant-pathology-2020-fgvc7/submit

In [ ]:
!pip install efficientnet

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
import os
import shutil
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os
import keras
from keras.engine import training
from keras.models import Model, Input
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Average
from keras.layers import Activation


In [ ]:
os.listdir('../input/tpu-trained-test')

In [ ]:
VGG16_WEIGHT_FILE = os.path.join(os.getcwd(), '../input/tpu-trained-test', 'vgg16(224).hdf5')
EFN_WEIGHT_FILE = os.path.join(os.getcwd(),'../input/tpu-trained-test','efn(224).hdf5')

In [ ]:
# MOBILE_NET_WEIGHT_FILE = os.path.join(os.getcwd(), '../input/pretrained-models', 'mobilenet_cnn.hdf5')
# VGG16_WEIGHT_FILE = os.path.join(os.getcwd(), '../input/pretrained-models', 'vgg16_cnn.hdf5')
# EFN_WEIGHT_FILE = os.path.join(os.getcwd(),'../input/pretrained-models','efn_cnn.hdf5')
# DENSE_NET_WEIGHT_FILE = os.path.join(os.getcwd(),'../input/pretrained-models','densenet_cnn.hdf5')

# Loading Data and Preprocessing

Here we load the data and take a look at what we're dealing with.

In [ ]:
train = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')


target = train[['healthy', 'multiple_diseases', 'rust', 'scab']]
test_ids = test['image_id']

train_len = train.shape[0]
test_len = test.shape[0]

train.describe()

Ah, we see the multiple_diseases label has drastically less images than the rest of the labels. Once we load the images in raw data form, we'll use scikitlearn to randomly over sample so we can fix this class imbalance.

Now let's load the image data.

In [ ]:
print("Shape of train data: " + str(train.shape))
print("Shape of test data: " + str(test.shape))

In [ ]:
train_len = train.shape[0]
test_len = test.shape[0]

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tqdm.notebook import tqdm

path = '../input/plant-pathology-2020-fgvc7/images/'
size = 224

train_images = np.ndarray(shape=(train_len, size, size, 3))
for i in tqdm(range(train_len)):
  img = load_img(path + f'Train_{i}.jpg', target_size=(size, size))
  train_images[i] = np.uint8(img_to_array(img))

test_images = np.ndarray(shape=(test_len, size, size, 3))
for i in tqdm(range(test_len)):
  img = load_img(path + f'Test_{i}.jpg', target_size=(size, size))
  test_images[i] = np.uint8(img_to_array(img))

train_images.shape, test_images.shape

Let's take a look at what the images look like.

In [ ]:
for i in range(4):
	plt.subplot(220 + 1 + i)
	plt.title(train['image_id'][i])
	plt.imshow(np.uint8(train_images[i]), interpolation = 'nearest', aspect='auto')
plt.show()
plt.savefig('train_images.png')

In [ ]:
for i in range(4):
	plt.subplot(220 + 1 + i)
	plt.title(test['image_id'][i])
	plt.imshow(np.uint8(test_images[i]), interpolation = 'nearest', aspect='auto')
plt.show()
plt.savefig('test_images.png')

Let's split out data into train and test sets for the model.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_images, target.to_numpy(), test_size=0.1, random_state=289) 

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train[1]

Now use RandomOverSampler to fix our class imbalance in the multiple diseases class.

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=289)

x_train, y_train = ros.fit_resample(x_train.reshape((-1, size * size * 3)), y_train)
x_train = x_train.reshape((-1, size, size, 3))
x_train.shape, y_train.shape

In [ ]:
import gc

del train_images
gc.collect()

Now we prepare the data for going into a Keras deep learning model. Here I use the ImageDataGenerator to also give us more images by using the parameters to rotate, horizontally flip, and vertically flip. Also the image is samplewise standard normalized the raw data so that the activation functions work properly.

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

batch_size = 8

train_datagen = ImageDataGenerator(samplewise_center = True,
                                   samplewise_std_normalization = True,
                                   horizontal_flip = True,
                                   shear_range=0.1,
                                   zoom_range=0.3,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   vertical_flip = True,
                                   brightness_range = [0.1,0.2],
                                   rotation_range=70)

train_generator = train_datagen.flow(
    x = x_train, 
    y = y_train,
    batch_size = batch_size)

validation_datagen = ImageDataGenerator(samplewise_center = True,
                                        samplewise_std_normalization = True)

validation_generator = validation_datagen.flow(
    x = x_test, 
    y = y_test,
    batch_size = batch_size)

Let's see what the images look like after processing and what they look like going into the model.

In [ ]:
idx = np.random.randint(8)
x, y = train_generator.__getitem__(idx)
plt.title(y[idx])
plt.imshow(x[idx])

# Keras Model
Here we build the model. I will use a pre-trained MobileNet for deep CNN which will then be fed into a dense layer to predict 4 classes, since the original MobileNet predicts 1000. It will compile using the loss function KL Divergence, Adam optimizer, and accuracy metric.

In [ ]:
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape,
                                                                                          x_test.shape, y_test.shape))

In [ ]:
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [ ]:
model_input

# MobileNet Model

In [ ]:
### TESTING
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average

def mobilenet_cnn(model_input: Tensor) -> training.Model:
    
    x = keras.applications.MobileNet(input_shape=(size, size, 3), weights='imagenet', include_top=False)(model_input)
#     x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(4, (1, 1))(x) # Four feature Maps
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='mobilenet_cnn')
    
    return model

In [ ]:
# def mobilenet_cnn(model_input: Tensor) -> training.Model:
#     pre_trained = keras.applications.MobileNet(input_shape=(size, size, 3), weights='imagenet', include_top=False)
#     for layer in pre_trained.layers:
#         layer.trainable = False
    
#     #pretrained_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(SIZE,SIZE,3), include_top=False)
#     mobilenetmodel = keras.Sequential([
#       pre_trained,
#       keras.layers.Flatten(),
#       keras.layers.GlobalAveragePooling2D(),
#       keras.layers.Dropout(0.3),
#       keras.layers.Dense(4,activation='softmax')
#       ])
#     return mobilenetmodel

In [ ]:
mobilenet_cnn_model = mobilenet_cnn(model_input)

In [ ]:
epochs = 15
steps_per_epoch = x_train.shape[0] // batch_size
validation_steps = x_test.shape[0] // batch_size
print(steps_per_epoch)

In [ ]:

es = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, verbose=1)
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,verbose=1,min_lr=0.000001,patience=6)


start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005
rampup_epochs = 40
sustain_epochs = 20
exp_decay = .8

def lrfn(epoch):
  if epoch < rampup_epochs:
    return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
  elif epoch < rampup_epochs + sustain_epochs:
    return max_lr
  else:
    return min_lr
    
lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

rang = np.arange(epochs)
y = [lrfn(x) for x in rang]
plt.plot(rang, y)
print('Learning rate per epoch:')

In [ ]:

def compile_and_train(model: training.Model, num_epochs: int) -> Tuple [History, str]: 
    
    
    model.compile(
        loss = 'categorical_crossentropy', 
        optimizer = 'adam', 
        metrics = ['accuracy'])
    
    filepath = '/kaggle/working/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
#     checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
#                                                  save_best_only=True, mode='auto', period=1)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True,mode='min')
    
    
    history = model.fit(
    x = train_generator,  
    validation_data = validation_generator,
    epochs = epochs,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    verbose=1,
    callbacks=[checkpoint,reduce_lr])
    
    
    weight_files = glob.glob(os.path.join(os.getcwd(), '/kaggle/working/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return history, weight_file

In [ ]:
_, mobilenet_cnn_weight_file = compile_and_train(mobilenet_cnn_model, 5)

In [ ]:
def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]    
    return error

In [ ]:
try:
    mobilenet_cnn_weight_file
except NameError:
    mobilenet_cnn_model.load_weights(MOBILE_NET_WEIGHT_FILE)
evaluate_error(mobilenet_cnn_model)

# Second Network

In [ ]:
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,Activation
def get_model():
    base_model = tf.keras.applications.VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(IMG_SIZE,IMG_SIZE, 3))
    x =  Conv2D(nb_classes, (1, 1))(base_model.output)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

In [ ]:
### TESTING
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average

def vgg16_cnn(model_input: Tensor) -> training.Model:
    
    x = keras.applications.VGG16(input_shape=(size, size, 3), weights='imagenet', include_top=False)(model_input)
#     x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(4, (1, 1))(x) # Four feature Maps
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='mobilenet_cnn')
    
    return model

In [ ]:
# def vgg16_cnn(model_input: Tensor) -> training.Model:
#     pre_trained = keras.applications.MobileNet(input_shape=(size, size, 3), weights='imagenet', include_top=False)
#     for layer in pre_trained.layers:
#         layer.trainable = False
    
#     #pretrained_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(SIZE,SIZE,3), include_top=False)
#     vgg16model = keras.Sequential([
#       pre_trained,
# #       keras.layers.Flatten(),
# #       keras.layers.Dropout(0.3),
#       keras.layers.GlobalAveragePooling2D(),
#       keras.layers.Dense(4, activation='softmax')
#       ])
#     return vgg16model

In [ ]:
vgg16_cnn_model = vgg16_cnn_tpu(model_input)

In [ ]:
vgg16_cnn_model.summary()

In [ ]:
_, vgg16_cnn_weight_file = compile_and_train(vgg16_cnn_model, 5)

In [ ]:
try:
    vgg16_cnn_weight_file
except NameError:
    vgg16_cnn_model.load_weights(VGG16_WEIGHT_FILE)
evaluate_error(vgg16_cnn_model)

# Efficient Net Model

In [ ]:
import efficientnet.keras as efn

In [ ]:
### TESTING
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average

def efn_cnn(model_input: Tensor) -> training.Model:
    
    x  = efn.EfficientNetB7(input_shape=(size, size, 3), weights='imagenet', include_top=False)(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(4, (1, 1))(x) # Four feature Maps
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='mobilenet_cnn')
    
    return model

In [ ]:
efn_cnn_model = efn_cnn(model_input)

In [ ]:
_, efn_cnn_weight_file = compile_and_train(efn_cnn_model, 5)

In [ ]:
try:
    efn_cnn_weight_file
except NameError:
    efn_cnn_model.load_weights(EFN_WEIGHT_FILE)
evaluate_error(efn_cnn_model)

# RESNET50

In [ ]:
### TESTING
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average

def resnet_cnn(model_input: Tensor) -> training.Model:
    
    x  = keras.applications.ResNet50(input_shape=(size, size, 3), weights='imagenet', include_top=False)(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(4, (1, 1))(x) # Four feature Maps
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='reset_cnn')
    
    return model

In [ ]:
resnet_cnn_model = resnet_cnn(model_input)

In [ ]:
_, resnet_cnn_weight_file = compile_and_train(resnet_cnn_model, 5)

In [ ]:
try:
    resnet_cnn_weight_file
except NameError:
    resnet_cnn_model.load_weights(DENSE_NET_WEIGHT_FILE)
evaluate_error(resnet_cnn_model)

# Ensemble

In [ ]:

mobilenet_cnn_model = mobilenet_cnn(model_input)
vgg16_cnn_model = vgg16_cnn(model_input)
efn_cnn_model = efn_cnn(model_input)
resnet_cnn_model = resnet_cnn(model_input)


try:
    mobilenet_cnn_model.load_weights(mobilenet_cnn_weight_file)
except NameError:
    mobilenet_cnn_model.load_weights(MOBILE_NET_WEIGHT_FILE)

try:
    vgg16_cnn_model.load_weights(vgg16_cnn_weight_file)
except NameError:
    vgg16_cnn_model.load_weights(VGG16_WEIGHT_FILE)
    
try:
    efn_cnn_model.load_weights(efn_cnn_weight_file)
except NameError:
    efn_cnn_model.load_weights(EFN_WEIGHT_FILE)
    
try:
    resnet_cnn_model.load_weights(resnet_cnn_weight_file)
except NameError:
    resnet_cnn_model.load_weights(DENSE_NET_WEIGHT_FILE)



models = [mobilenet_cnn_model, vgg16_cnn_model,efn_cnn_model,resnet_cnn_model]

In [ ]:
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
ensemble_model = ensemble(models, model_input)

In [ ]:
evaluate_error(ensemble_model)

In [ ]:
pair_A = [efn_cnn_model, resnet_cnn_model]

In [ ]:
pair_A_ensemble_model = ensemble(pair_A, model_input)
evaluate_error(pair_A_ensemble_model)

# End of Proj

# Weighted Ensemble

In [ ]:
from numpy import array


def ensemble_predictions(models: List [training.Model], model_input: Tensor) -> training.Model:
	# make predictions
	outputs = [model.outputs[0] for model in models]
	y = array(outputs)
	# sum across ensemble members
	summed = numpy.sum(y, axis=0)
	# argmax across classes
	result = argmax(summed, axis=1)
    model = Model(model_input, result, name='ensemble')
    
    
	return model


# def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
#     outputs = [model.outputs[0] for model in models]
#     y = Average()(outputs)
    
#     model = Model(model_input, y, name='ensemble')
    
#     return model


In [ ]:
f

In [ ]:
gdgdgdd

In [ ]:
fwefwefwffeeffewwqweffffffffffffff

In [ ]:
afasfasfsfsfsfsfs

In [ ]:
# # Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper right')
# plt.show()

In [ ]:
# train_err = (1-history.history['accuracy'][-1])*100
# validation_err = (1-history.history['val_accuracy'][-1])*100
# print("Train set error " + str(train_err))
# print("Validation set error " + str(validation_err))

In [ ]:
test_datagen = ImageDataGenerator(samplewise_center = True,
                                   samplewise_std_normalization = True,
                                   horizontal_flip = True,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   vertical_flip = True,
                                   brightness_range = [0.1,0.2],
                                   rotation_range=70)

test_generator = test_datagen.flow(
    x = test_images,
    shuffle = False)

In [ ]:
probabilities = pair_A_ensemble_model.predict(test_generator, steps = len(test_generator))
print(probabilities)
print(probabilities[:,0].mean()*100)
print(probabilities[:,1].mean()*100)
print(probabilities[:,2].mean()*100)
print(probabilities[:,3].mean()*100)

In [ ]:
res = pd.DataFrame()
res['image_id'] = test['image_id']
res['healthy'] = probabilities[:, 0]
res['multiple_diseases'] = probabilities[:, 1]
res['rust'] = probabilities[:, 2]
res['scab'] = probabilities[:, 3]

In [ ]:
res.to_csv('submission_pairA.csv', index=False)

In [ ]:
valid_probabilities = mobilenet_cnn_model.predict(validation_generator, steps = len(validation_generator))
print(valid_probabilities[:,0].mean()*100)
print(valid_probabilities[:,1].mean()*100)
print(valid_probabilities[:,2].mean()*100)
print(valid_probabilities[:,3].mean()*100)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, valid_probabilities)